In [2]:
import xarray as xr
import distributed


In [2]:
cluster = distributed.LocalCluster(n_workers=4, threads_per_worker=1, local_directory='/tmp')
client = distributed.Client(cluster)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 4,Total memory: 15.34 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:53791,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 4
Started: Just now,Total memory: 15.34 GiB
Comm: tcp://127.0.0.1:53814,Total threads: 1
Dashboard: http://127.0.0.1:53815/status,Memory: 3.84 GiB
Nanny: tcp://127.0.0.1:53794,


# Load IFS data

In [3]:
ds_ifs_pl = xr.open_dataset(r'D:\Projects\NIOT\v1\ensemble_transformer\data\pl_201701.nc',chunks={'time': 2})
ds_ifs_sfc = xr.open_dataset(r'D:\Projects\NIOT\v1\ensemble_transformer\data\sfc_202312.nc', chunks={'time': 2})

In [4]:
ds_ifs_pl

<xarray.Dataset>
Dimensions:    (latitude: 6, level: 2, longitude: 6, number: 10, time: 62)
Coordinates:
  * longitude  (longitude) float32 68.0 73.62 79.25 84.88 90.5 96.12
  * latitude   (latitude) float32 34.12 28.5 22.88 17.25 11.62 6.0
  * level      (level) int32 500 850
  * number     (number) int32 1 2 3 4 5 6 7 8 9 10
  * time       (time) datetime64[ns] 2017-01-03 ... 2017-02-02T12:00:00
Data variables:
    t          (time, number, level, latitude, longitude) float32 dask.array<chunksize=(2, 10, 2, 6, 6), meta=np.ndarray>
    gh         (time, number, level, latitude, longitude) float32 dask.array<chunksize=(2, 10, 2, 6, 6), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    history:      2024-06-13 10:32:16 GMT by grib_to_netcdf-2.33.2: /opt/ecmw...

In [5]:
ds_ifs_gh = ds_ifs_pl['gh'].to_dataset("level")
ds_ifs_gh = ds_ifs_gh.rename({500: 'gh_500', 850: 'gh_850'})

In [5]:
ds_ifs_pl

<xarray.Dataset>
Dimensions:    (latitude: 6, level: 2, longitude: 6, number: 10, time: 62)
Coordinates:
  * longitude  (longitude) float32 68.0 73.62 79.25 84.88 90.5 96.12
  * latitude   (latitude) float32 34.12 28.5 22.88 17.25 11.62 6.0
  * level      (level) int32 500 850
  * number     (number) int32 1 2 3 4 5 6 7 8 9 10
  * time       (time) datetime64[ns] 2017-01-03 ... 2017-02-02T12:00:00
Data variables:
    t          (time, number, level, latitude, longitude) float32 dask.array<chunksize=(2, 10, 2, 6, 6), meta=np.ndarray>
    gh         (time, number, level, latitude, longitude) float32 dask.array<chunksize=(2, 10, 2, 6, 6), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    history:      2024-06-13 10:32:16 GMT by grib_to_netcdf-2.33.2: /opt/ecmw...

In [7]:
ds_ifs_t = ds_ifs_pl['t'].to_dataset("level")
ds_ifs_t = ds_ifs_t.rename({500: 't_500', 850: 't_850'})

In [8]:
ds_ifs_t['longitude']

<xarray.DataArray 'longitude' (longitude: 1)>
array([78.], dtype=float32)
Coordinates:
  * longitude  (longitude) float32 78.0
Attributes:
    units:      degrees_east
    long_name:  longitude

In [9]:
ds_ifs_merged = xr.merge([ds_ifs_sfc, ds_ifs_t, ds_ifs_gh])

In [10]:
ds_ifs_merged = ds_ifs_merged.isel(latitude=slice(None, None, -1))
ds_ifs_merged = ds_ifs_merged.rename({'number': 'ensemble'})

In [11]:
ds_ifs_merged

<xarray.Dataset>
Dimensions:    (ensemble: 10, latitude: 1, longitude: 1, time: 62)
Coordinates:
  * longitude  (longitude) float32 78.0
  * latitude   (latitude) float32 10.0
  * ensemble   (ensemble) int32 1 2 3 4 5 6 7 8 9 10
  * time       (time) datetime64[ns] 2023-12-03 ... 2024-01-02T12:00:00
Data variables:
    t2m        (time, ensemble, latitude, longitude) float32 dask.array<chunksize=(2, 10, 1, 1), meta=np.ndarray>
    t_500      (time, ensemble, latitude, longitude) float32 dask.array<chunksize=(2, 10, 1, 1), meta=np.ndarray>
    t_850      (time, ensemble, latitude, longitude) float32 dask.array<chunksize=(2, 10, 1, 1), meta=np.ndarray>
    gh_500     (time, ensemble, latitude, longitude) float32 dask.array<chunksize=(2, 10, 1, 1), meta=np.ndarray>
    gh_850     (time, ensemble, latitude, longitude) float32 dask.array<chunksize=(2, 10, 1, 1), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    history:      2024-06-13 05:16:05 GMT by grib_to_netcdf-2.33.2: /opt/ecmw...

In [11]:
ds_ifs_train = ds_ifs_merged.sel()
ds_ifs_train.to_zarr(
    '../data/processed/ifs/ds_train',
    encoding={
        't2m': {'dtype': 'float32', 'scale_factor': 1.0, 'add_offset': 0.0},
        't_500': {'dtype': 'float32', 'scale_factor': 1.0, 'add_offset': 0.0},
        't_850': {'dtype': 'float32', 'scale_factor': 1.0, 'add_offset': 0.0},
        'gh_500': {'dtype': 'float32', 'scale_factor': 1.0, 'add_offset': 0.0},
        'gh_850': {'dtype': 'float32', 'scale_factor': 1.0, 'add_offset': 0.0},
    }
)

ContainsGroupError: path '' contains a group

In [12]:
ds_ifs_train

<xarray.Dataset>
Dimensions:    (ensemble: 10, latitude: 1, longitude: 1, time: 62)
Coordinates:
  * longitude  (longitude) float32 78.0
  * latitude   (latitude) float32 10.0
  * ensemble   (ensemble) int32 1 2 3 4 5 6 7 8 9 10
  * time       (time) datetime64[ns] 2023-12-03 ... 2024-01-02T12:00:00
Data variables:
    t2m        (time, ensemble, latitude, longitude) float32 dask.array<chunksize=(2, 10, 1, 1), meta=np.ndarray>
    t_500      (time, ensemble, latitude, longitude) float32 dask.array<chunksize=(2, 10, 1, 1), meta=np.ndarray>
    t_850      (time, ensemble, latitude, longitude) float32 dask.array<chunksize=(2, 10, 1, 1), meta=np.ndarray>
    gh_500     (time, ensemble, latitude, longitude) float32 dask.array<chunksize=(2, 10, 1, 1), meta=np.ndarray>
    gh_850     (time, ensemble, latitude, longitude) float32 dask.array<chunksize=(2, 10, 1, 1), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    history:      2024-06-10 09:34:24 GMT by grib_to_netcdf-2.33.2: /opt/ecmw...

In [9]:
ds_ifs_test = ds_ifs_merged.sel()
ds_ifs_test.to_zarr(
    '../data/processed/ifs/ds_test',
    encoding={
        't2m': {'dtype': 'float32', 'scale_factor': 1.0, 'add_offset': 0.0},
        't_500': {'dtype': 'float32', 'scale_factor': 1.0, 'add_offset': 0.0},
        't_850': {'dtype': 'float32', 'scale_factor': 1.0, 'add_offset': 0.0},
        'gh_500': {'dtype': 'float32', 'scale_factor': 1.0, 'add_offset': 0.0},
        'gh_850': {'dtype': 'float32', 'scale_factor': 1.0, 'add_offset': 0.0},
    }
)